In [107]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, load_prompt
from langchain.schema import StrOutputParser

from langchain.prompts.pipeline import PipelinePromptTemplate

### Model

In [108]:
model = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    api_key=open('api.txt', 'r').read()
)

### Prompts

In [109]:
full_temp = """{init}

{task}

{best_practice}"""

full_prompt = PromptTemplate.from_template(full_temp)

In [110]:
# init_prompt = load_prompt('prompts/init_prompt.json').format(role="Systems Engineer")
# task_prompt = load_prompt('prompts/task_prompt.json').format(ticket="title: The color from the button is not displayed correctly when i use a different browser; description: When i press the save button the color blue is displayed instead of green. This is only when i use Chrome as browser. When i use Firefox it works fine.")
# sumLen_prompt = load_prompt('prompts/sumLen_prompt.json').format(min=6,max=11)
init_prompt = load_prompt('prompts/init_prompt.json')
task_prompt = load_prompt('prompts/task_prompt.json')
sumLen_prompt = load_prompt('prompts/sumLen_prompt.json')

In [111]:
input_prompts = [
    ("init", init_prompt),
    ("task", task_prompt),
    ("best_practice", sumLen_prompt)
]

pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=input_prompts)

In [112]:
final_prompt = pipeline_prompt.format(
    role="Systems Engineer",
    ticket="title: The color from the button is not displayed correctly when i use a different browser; description: When i press the save button the color blue is displayed instead of green. This is only when i use Chrome as browser. When i use Firefox it works fine.",
    min=6,
    max=11
)
print(final_prompt)

Act as a Systems Engineer working with issue trackers like Jira or Azure DevOps. There are multiple text-based best practices you have to take into account.

Check this ticket (title: The color from the button is not displayed correctly when i use a different browser; description: When i press the save button the color blue is displayed instead of green. This is only when i use Chrome as browser. When i use Firefox it works fine.) to see if these best practices are violated, and if so, provide a recommendation that satisfies the best practices.

The length of the title of a ticket should be between 6 and 11 words. Return only the new title and nothing else: 


### Parser

In [113]:
model_parser = model | StrOutputParser()

### Chain

### Run

In [115]:
result = model.invoke(final_prompt)
result.content

'Button color not displayed correctly in Chrome browser'